## this is the second example: param linear wave equation

In [ ]:
# periodic bc wave equations
import tensorflow as tf
#import pandas as pd
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import sys
import time
import os
import h5py


In [ ]:
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense

In [ ]:
tf.keras.utils.set_random_seed(1) 
tf.config.experimental.enable_op_determinism()


dtype_=tf.float64
tf.keras.backend.set_floatx('float64')

In [ ]:
#If there are GPUs, the first one is used by default
gpus = tf.config.list_physical_devices('GPU')
print("Available GPUs:",gpus)
tf.config.set_visible_devices(gpus[0], 'GPU')
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH']="true"   

In [ ]:
curr_path = os.getcwd()   # the current directory
print(curr_path)

print(tf.__version__)

### building the model

In [ ]:



# the optional block of our symplectic NN, G-reflector
class SympMatLayer(layers.Layer):
    def __init__(self, nif):
        super(SympMatLayer, self).__init__()
        init = tf.initializers.GlorotNormal(seed=0)
        #init_zero = tf.zeros_initializer()

        self.nif = nif
        # beta_init = init(shape=[1,1], dtype = dtype_)
        # uvec_init = init(shape=[2*self.dim,1], dtype = dtype_)

        self.beta = self.add_weight(name = 'beta', shape=[1,1], initializer=init, trainable=True)

        self.uvec = self.add_weight(name = 'u', shape=[2*self.nif,1], initializer=init, trainable=True)


    @tf.function
    def call(self,z):
        zJt = tf.concat([z[:,self.nif:],-z[:,0:self.nif]], axis=1)
        out = z + self.beta*tf.matmul(tf.matmul(zJt, self.uvec), tf.transpose(self.uvec))
        return out

    '''
        Note the inverse of G is
        G^{-1} = I - beta u u^T J
        Comment: this is a nice property!
    '''
    @tf.function
    def inverse(self, z):
        zJt = tf.concat([z[:,self.nif:],-z[:,0:self.nif]], axis=1)
        out = z - self.beta*tf.matmul(tf.matmul(zJt, self.uvec), tf.transpose(self.uvec))
        return out
    

# great symp matrix with G-reflectors

class SympMat(Model):
    """
    N: number of G-reflectors
    nif: dim of full physical space
    nir: dim of reduced latent space
    """
    def __init__(self,N,nif,nir):
        super(SympMat, self).__init__()
        self.N = N
        self.nif = nif
        self.nir = nir
        self.hlayers=[]
        for i in range(self.N):
            hl = SympMatLayer(self.nif)
            self.hlayers.append(hl)


        self.paddings = tf.constant([[0,0], [0,self.nif-nir]])


    @tf.function
    def project(self, r):
        rout = r
        for i in range(self.N):
            rout = self.hlayers[i](rout)

        return tf.concat([rout[:,:self.nir],rout[:,self.nif:(self.nif+self.nir)]], axis=1)

    @tf.function
    def inverse(self, r):
        # K = r.shape[0]
        # d = self.nif-self.nir

        q = r[:,:self.nir]
        q = tf.pad(q, self.paddings, "CONSTANT")
        p = r[:,self.nir:]
        p = tf.pad(p, self.paddings, "CONSTANT")

        rout = tf.concat([q,p], axis=1)
        for i in range(self.N)[-1::-1]:
            rout = self.hlayers[i].inverse(rout)

        return rout


    @tf.function
    def call(self, r):
        rout = r

        # mask = np.ones_like(r.numpy())
        # mask[:,self.nir:self.nif] = 0.
        # mask[:,(self.nif+self.nir):] = 0.

        # mask = tf.constant(mask, dtype=tf.float64)

        for i in range(self.N):
            rout = self.hlayers[i](rout)

        # print( tf.concat([rout[:,:self.nir], tf.zeros_like(rout)[:,:(self.nif-self.nir)],rout[:,self.nif:(self.nif+self.nir)]], axis=1))
        # rout = rout*mask

        rout = tf.concat([rout[:,:self.nir], tf.zeros_like(rout)[:,:(self.nif-self.nir)],
                          rout[:,self.nif:(self.nif+self.nir)],
                          tf.zeros_like(rout)[:,:(self.nif-self.nir)]
                          ], axis=1)

        for i in range(self.N)[-1::-1]:
            rout = self.hlayers[i].inverse(rout)



        return rout
    

In [ ]:
# the main block of our symplectic NN, HenonNets
#potential function
class Potential(layers.Layer):
    def __init__(self, units=5):
        super(Potential, self).__init__()
        self.units = units
        self.dense_1 = Dense(name='potential.1', units = self.units, activation='elu') # layer class Linear has to be defined before
        self.dense_2 = Dense(name='potential.2', units = self.units, activation='elu')
        self.out = Dense(1,use_bias=False) # output size is 1, no bias

    def call(self, inputs):
        x = self.dense_1(inputs)
        x = self.dense_2(x)
        return self.out(x)


ymean_tf = tf.constant(0, dtype=tf.float64)
ydiam_tf = tf.constant(1., dtype=tf.float64)


class HenonLayer(layers.Layer):
    def __init__(self, units=10):
        super(HenonLayer, self).__init__()
        self.units = units
        self.V = Potential(self.units)

    def build(self, input_shape): # the input_shape is the shape of z in the call function
        self.eta = self.add_weight(name = 'eta', shape=(input_shape[-1]//2,), trainable=True)

    def call(self, z, inverse = False):
        if inverse:
            return self.inverse(z)
        return self.forward(z)

    @tf.function
    def forward(self, z):
        #ipdb.set_trace()
        ni = int(z.shape[-1]/2)
        X = z[:,:ni]
        Y = z[:,ni:2*ni]
        kw = z[:,-1:]
        for k in range(4):
            with tf.GradientTape() as tape:
                tape.watch(Y)
                Ylast = (tf.concat([Y,kw],axis=1) - ymean_tf) / ydiam_tf
                p = self.V(Ylast)

            Yout = -X + tape.gradient(p, Y)
            X = Y + self.eta
            Y = Yout

        return tf.concat([X,Y,kw], axis = 1)#batch*(2n+1)
    @tf.function
    def inverse(self, z): # compute the inverse of the HenonLayer

        ni = int(z.shape[-1]/2)
        X = z[:,:ni]
        Y = z[:,ni:2*ni]
        kw = z[:,-1:]
        for k in range(4):
            X = X - self.eta
            with tf.GradientTape() as tape:
                tape.watch(X)
                Xlast = (X - ymean_tf)/ydiam_tf
                p = self.V(tf.concat([Xlast,kw],axis=1))
            Ylast = X
            X = -Y + tape.gradient(p, X)
            Y = Ylast

        return tf.concat([X,Y,kw], axis = 1)




In [ ]:
class SympMatROM(Model):
    """
        units_list: the list of

    """
    def __init__(self, units_list, N, nif,nir):
        super(SympMatROM, self).__init__()
        self.loss_tracker = tf.keras.metrics.Mean(name="loss")
        self.N = N
        self.nif = nif
        self.nir = nir
        self.matlayers = []
        self.henonlayers = []

        for i in units_list:
            hl = HenonLayer(i)
            self.henonlayers.append(hl)

        for i in range(self.N):
            ml = SympMatLayer(self.nif)
            self.matlayers.append(ml)


        self.paddings = tf.constant([[0,0], [0,self.nif-self.nir]])


    @tf.function
    def project(self, r):
        rout = r
        for i in range(len(self.henonlayers)):
            rout = self.henonlayers[i](rout)
        for i in range(self.N):
            rout = self.matlayers[i](rout)

        return tf.concat([rout[:,:self.nir],rout[:,self.nif:(self.nif+self.nir)]], axis=1)

    @tf.function
    def embedding(self, r):
        # K = r.shape[0]
        # d = self.nif-self.nir

        q = r[:,:self.nir]
        q = tf.pad(q, self.paddings, "CONSTANT")
        p = r[:,self.nir:]
        p = tf.pad(p, self.paddings, "CONSTANT")

        rout = tf.concat([q,p], axis=1)
        for i in range(self.N)[-1::-1]:
            rout = self.matlayers[i].inverse(rout)

        for i in range(len(self.henonlayers))[-1::-1]:
            rout = self.henonlayers[i].inverse(rout)

        return rout


    @tf.function
    def call(self, r):
        rout = r


        rout = self.project(rout)
        rout = self.embedding(rout)



        return rout

    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
           
            loss = self.compute_loss(y=y, y_pred=y_pred)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        # need to update the loss in loss_tracker
        self.loss_tracker.update_state(loss) # Add current batch loss

        # Return a dict mapping metric names to current value
        return {"loss": self.loss_tracker.result()}




    def test_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data


        y_pred = self(x, training=False)  # Forward pass
        loss = self.compute_loss(y=y, y_pred=y_pred)
        self.loss_tracker.update_state(loss) # Add current batch loss


        # Return a dict mapping metric names to current value
        return {"loss": self.loss_tracker.result()}
        # return {"mae": self.mae_metric.result()}


In [ ]:
def hamiltonian(z):
  """
  compute the discrete hamiltonian, it depends on the concrete equation, and the physical space; aiming at computing the hamiltonian loss
  """
  kw = z[:,-1:] # wave speed
  delta_x = 1/200 # grid spacing
  q = z[:,:z.shape[-1]//2]# q
  p = z[:,z.shape[-1]//2:z.shape[-1]//2*2]# p
  q_r = tf.concat([q[:,1:],q[:,0:1]],axis=1)# q1-qN+1
  q_l = tf.concat([q[:,-1:],q[:,:-1]],axis=1)# q0-qN-1

  h = delta_x/2*tf.reduce_sum(p**2+kw*(q_r-q)**2/(2*delta_x**2)+kw*(q-q_l)**2/(2*delta_x**2),keepdims=True,axis=1)
  return h



In [ ]:
class SympMatFlowMappingROM(Model):
    """
        units_list: the list of 

    """
    def __init__(self, units_list_ae, N, nif, nir, units_list_fm, n, n_train_time,lamb1):
        super(SympMatFlowMappingROM, self).__init__()
        self.loss_tracker = tf.keras.metrics.Mean(name="loss")
        self.N = N
        self.nif = nif
        self.nir = nir
        self.matlayers_ae = []
        self.henonlayers_ae = []
        self.n = n
        self.matlayers_fm = []
        self.henonlayers_fm = []
        self.n_train_time = n_train_time
        self.lamb1 = lamb1 # balanced hamilton loss

        for i in units_list_ae:
            hl = HenonLayer(i)
            self.henonlayers_ae.append(hl)

        for i in range(self.N):
            ml = SympMatLayer(self.nif)
            self.matlayers_ae.append(ml)

        for i in units_list_fm:
            hl = HenonLayer(i)
            self.henonlayers_fm.append(hl)

        for i in range(self.n):
            ml = SympMatLayer(self.nir)
            self.matlayers_fm.append(ml)

        self.paddings = tf.constant([[0,0], [0,self.nif-self.nir]])

     
    @tf.function
    def project(self, r):
        rout = r# forward  batch*(2n+1)
        for i in range(len(self.henonlayers_ae)):
            rout = self.henonlayers_ae[i](rout) #batch*(2n+1)
        rout1 = rout[:,:2*self.nif]
        for i in range(self.N):
            rout1 = self.matlayers_ae[i](rout1)#batch*2n

        return tf.concat([rout1[:,:self.nir],rout1[:,self.nif:(self.nif+self.nir)],rout[:,-1:]], axis=1) #batch*(2nir+1),the last dim is kw
            
    @tf.function
    def embedding(self, r):
        # K = r.shape[0]
        # d = self.nif-self.nir

        q = r[:,:self.nir]
        q = tf.pad(q, self.paddings, "CONSTANT")
        p = r[:,self.nir:2*self.nir]
        p = tf.pad(p, self.paddings, "CONSTANT")
        kw = r[:,-1:]
        rout = tf.concat([q,p], axis=1)
        for i in range(self.N)[-1::-1]:
            rout = self.matlayers_ae[i].inverse(rout)
        rout1 = tf.concat([rout,kw], axis=1)#batch*(2n+1)
        for i in range(len(self.henonlayers_ae))[-1::-1]:
            rout1 = self.henonlayers_ae[i].inverse(rout1)

        return rout1

    @tf.function
    def flowmapping(self, r):

        q = r[:,:self.nir]
        p = r[:,self.nir:2*self.nir]
        kw = r[:,-1:]
        rout = tf.concat([q,p], axis=1)
        for i in range(self.n):
            rout = self.matlayers_fm[i](rout)
        rout1 = tf.concat([rout,kw],axis=1)#2*nir+1
        for i in range(len(self.henonlayers_fm)):
            rout1 = self.henonlayers_fm[i](rout1)#2*nir+1

        return rout1
    
    def inverse(self,r):
        rout = r
        for i in range(len(self.henonlayers_fm))[-1::-1]:
            rout = self.henonlayers_fm[i].inverse(rout)#2*nir+1
        rout1 = rout[:,:2*self.nir]
        for i in range(self.n)[-1::-1]:
            rout1 = self.matlayers_fm[i].inverse(rout1)
        return tf.concat([rout1,rout[:,-1:]], axis=1)



    @tf.function
    def call(self, r):
        rout = r

        rout = self.project(rout)
        rout = self.embedding(rout)
        
        return rout

    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`. #x batch*401
        x1, y = data
        x = x1[:,:self.nif*2]#batch*400
        # kw = x1[:,-1:]#batch*1


        with tf.GradientTape() as tape:
            z = self.project(x1)#2nir+1
            y0 = self.embedding(z)#2n+1
            ham_cons = hamiltonian(x)
            loss_enc = self.compute_loss(y=x, y_pred=y0[:,:-1])
            loss_ham = self.compute_loss(y=ham_cons, y_pred = hamiltonian(y0))

            for i in range(self.n_train_time):
                z = self.flowmapping(z)  # Forward pass 2nir+1
               
                y0 = self.embedding(z) #2n+1
                loss_enc += self.compute_loss(y=y[:,i,:], y_pred=y0[:,:-1])
                loss_ham += self.compute_loss(y=ham_cons, y_pred = hamiltonian(y0))
            loss = loss_enc + self.lamb1*loss_ham

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        # need to update the loss in loss_tracker
        self.loss_tracker.update_state(loss) # Add current batch loss

        # Return a dict mapping metric names to current value
        return {"loss": self.loss_tracker.result()}


### load data and train

In [ ]:
# parameters
c2 = 0.1
w_all = np.linspace(0,1,5)
k_w = np.zeros((5,5,5,5))
for i in range(5):
    for j in range(5):
        for k in range(5):
            for p in range(5):
                k_w[i,j,k,p] = c2*(w_all[i]+w_all[j]/4+w_all[k]/9+w_all[p]/16)
k_w = k_w.reshape(-1,1)
k_w1 = np.repeat(k_w,repeats = 44, axis=1)
k_w1 = k_w1.T
k_w1 = k_w1.reshape(-1,1)
k_w1.shape

In [ ]:
nlayer_ae = 2
nwidth_ae = 512

nlayer_fm = 2
nwidth_fm = 128

example_time_steps = 6


#activiation = 'relu'
batch_size = 256
epoch = 5000

initial_learning_rate = 1e-3
decay_steps=13 ## decay for every three epoch
decay_rate=0.99

n_x = 200 # data dimension
n_sample = 625


nif = n_x # dimension of full space
nir = 10 # dimension of reduced space

nref = nir # number of reflectors

verbose = 2

data_str = "Wave_bump_peri"

data_features_str = f"Wave_bump_peri_features_c_0.1_Nx_{n_x}_sample_{n_sample}_timesteps_44_dt_3.0"
data_labels_str = f"Wave_bump_peri_labels_c_0.1_Nx_{n_x}_sample_{n_sample}_timesteps_44_dt_3.0"

para_str = data_str + f"_nlayer_ae_{nlayer_ae}_hidden_ae_{nwidth_ae}"
para_str = para_str + f"_nlayer_fm_{nlayer_fm}_hidden_fm_{nwidth_fm}"
para_str = para_str + f"_nreflector_{nref}_lr_{initial_learning_rate}_decaystep_{decay_steps}"
para_str = para_str + f"_decayrate_{decay_rate}_batch_{batch_size}_epoch_{epoch}"


data_features_path = os.path.join(curr_path, "data", data_features_str + ".npy")
data_labels_path = os.path.join(curr_path, "data", data_labels_str + ".npy")

train_path = os.path.join(curr_path, "train", para_str + ".log")
check_path = os.path.join(curr_path, "check", para_str)


data_features = np.load(data_features_path)
data_labels = np.load(data_labels_path)






In [ ]:
features1 = np.concatenate([data_features,k_w1],axis=1)
# features1.shape
# here train for the autoencoder
dataset = tf.data.Dataset.from_tensor_slices((features1, data_labels))
dataset = dataset.shuffle(len(dataset)) # must include the shuffle size

DATASET_SIZE = len(dataset)

train_size = int(1.0 * DATASET_SIZE)

train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)



train_dataset = train_dataset.shuffle(len(train_dataset))
train_dataset = train_dataset.batch(batch_size)

test_dataset = test_dataset.batch(batch_size)


In [ ]:
# balance the reconstruction loss and hamiltonian loss
lamb1 = 0.01
if lamb1!=0:
    para_str = para_str + f"_lamb1_{lamb1}"

    


In [ ]:
directory = os.path.dirname(train_path)

# if not exist, create the directory
if not os.path.exists(directory):
    os.makedirs(directory)
with open(train_path, 'a') as f:

    f.write('log_file_name:  {:s} \n'.format(train_path))

In [ ]:


loss_fun = keras.losses.MeanSquaredError()
test_model = SympMatFlowMappingROM([nwidth_ae]*nlayer_ae,nref,nif,nir,[nwidth_fm]*nlayer_fm,nref,example_time_steps,lamb1)
# test_model = SympMatFlowMappingROM([nwidth_ae]*nlayer_ae,nir,nif,nir,[nwidth_fm]*nlayer_fm,nir,example_time_steps)

reload = False # reload the weights, If it is the first time training, it needs to be set to false
train = True # train or test the model


batch_number = train_size//batch_size
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    # 1e-4,
    decay_steps=batch_number*decay_steps,
    decay_rate=decay_rate,
    staircase=True)
Adamoptimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule)
# Train from a checkpt vs start a new training
checkpoint_path = os.path.join(curr_path, "check", para_str,"cp-{epoch:05d}.ckpt")
if reload:
    checkpoint_dir = os.path.dirname(checkpoint_path)
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    print('Load the previous training. The old model is in:')
    print(latest)
    epochs=latest.replace(check_path+'/', '')
    epochs=epochs.replace('cp-', '')
    epochs=epochs.replace('.ckpt', '')
    epoch0=4080
    print("Epoch0 = %5d" % epoch0)
    test_model.load_weights(latest)
else:
    epoch0=0
    print('start training from the begining...')
test_model.compile(optimizer = Adamoptimizer, loss = loss_fun)

# Create a callback that saves the model's weights every 10 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, verbose=verbose, save_weights_only=True, period=40)




In [ ]:
checkpoint_dir = os.path.dirname(checkpoint_path)
# if not exist, create the file
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [ ]:
class LearningRatePrintingCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        tf.print(
           'lr: ',test_model.optimizer.lr
            )





if train:
    # Train the model
    print('training model...')
    h = test_model.fit(train_dataset, epochs = epoch,
                    #    validation_data=test_dataset,
                       verbose=verbose, initial_epoch=epoch0,
                       callbacks = [cp_callback,LearningRatePrintingCallback()]) #
    print('training complete...')

In [ ]:
#test the reconstruction ability of the autoencoder
data_rec = test_model(features1)

rec_error = tf.reduce_mean(tf.square(features1[:,:-1]-data_rec[:,:-1]))
print(rec_error)

### test the model

In [ ]:
data_features1 = features1.reshape(44,625,401)
data_labels1 = data_labels.reshape(44,625,6,400)[-1].transpose(1,0,2)
k_w2 = np.stack([k_w]*6,axis=0)
data_labels1 = np.concatenate([data_labels1,k_w2],axis=2)
data_features_plus = np.concatenate([data_features1,data_labels1],axis=0)

In [ ]:
data_features_low = test_model.project(data_features_plus.reshape(-1,401))
data_features_low.shape

In [ ]:
data_low = data_features_low.numpy().reshape(50,625,21)


In [ ]:
#latent dynamics
times = np.linspace(0,3,50)
plt.figure(figsize=(8,6))
for i in range(10):
    plt.plot(times,data_low[:,24,i])
plt.xlabel('Time')
plt.ylabel('Magnitude')
plt.title('Latent Space - 1D linear wave')
# plt.plot(times,x2)

In [ ]:
#  time evolution, here you can choose the final time(delta t* time_step), for this example, delta t is chosen as 0.06
time_step = 49
ham_cons = []
x_test = data_features_plus[0,:,:]#625*401
ham_cons.append(hamiltonian(x_test))
z_test = test_model.project(x_test)#625*21
for i in range(time_step):
    
    z_test = test_model.flowmapping(z_test)#625*21
    
    y_test = test_model.embedding(z_test)#625*401
    ham_cons.append(hamiltonian(y_test))
y_test = test_model.embedding(z_test)

In [ ]:
ham_cons = tf.concat(ham_cons,axis=1)
ham_cons.shape

In [ ]:
#choose one trajectory and plot the trajectory, 
index = 24
plt.figure(figsize=(8,6))
x = np.linspace(0, 1, n_x)
# sparse plot
sparse_step = 5  
# plt.plot(x, y_test[24, :n_x], c='g', marker='o', markersize=6, label='Sym', markevery=sparse_step)
plt.plot(x, y_test[index, :n_x], c='g', marker='o', markersize=6, label='Sym', markevery=sparse_step)
plt.plot(x, data_features_plus[49, index, :n_x], c='b', linestyle='-', label='Exact')
plt.xlabel('x')
plt.ylabel('Magnitude')
plt.legend(loc='best')
# plt.title('Relative Error:')
plt.title('Recovered solution')
plt.show()

In [ ]:
#choose one trajectory, show the evolution of hamiltonian 

index = 100
ham_label = hamiltonian(data_features_plus[:,index])

times = np.linspace(0, 3, 50)
plt.figure(figsize=(8,6))
plt.plot(times,ham_cons[index],c='g', marker='o', markersize=6, label='Sym')
plt.plot(times,ham_label,c='b', linestyle='-', label='Exact')
# plt.plot(times,ham_cons[2,1:],c='y',label='Sample = 2')
plt.xlabel('Time')
# plt.ylabel('Magnitude')
plt.legend(loc='best')
plt.ylim(0,0.4)
# plt.title('Relative Error:')
plt.title('Hamiltonian')
plt.ylabel('H(t)')
plt.show()